# MicroHapulator: Interactive Demo

<small>Daniel Standage, 2022-01-12</small>

**MicroHapulator** is an application for empirical haplotype calling, analysis, and basic forensic interpretation of microhaplotypes with NGS data.
This notebook is designed to introduce MicroHapulator to new users and demonstrate its features.
The software is normally run by entering commands in a shell terminal window.
For convenience, however, this notebook provides an interactive environment that interleaves narrative text, shell commands that the reader can execute and re-execute, the output of those commands, and additional explanatory commentary.
To execute a block of code in the notebook, select the corresponding notebook cell and then click the `[> Run]` button at the top of the page (or as a keyboard shortcut, simultaneously press `[shift]` and `[enter]`).

## Overview

> *This demo assumes the reader is familiar with basic terminology and concepts related to biology, genomes, and NGS sequencing.
> A [primer on forensic DNA typing](https://microhapulator.readthedocs.io/en/latest/config.html) is available for the interested reader.*

MicroHapulator calls haplotypes by probing NGS reads aligned to a reference sequence for the corresponding marker.
Consider the following mock example.
The first line shows the reference sequence for the `mh01USC-1pD` marker.
Each subsequent line represents an NGS read aligned to the marker sequence.
The `*` symbols denote the locations of the SNPs present in the marker, and the `.` symbols denote locations where the NGS read matches the reference.

```
                       *                     *           *
AAATAGCTGGGCTAATAATGAACTGAAGCAAAGTCAACTGAAATGTCCTGGGCAGCTCCAGAAACTCCAGAATGGGGAGGA
.......................C.....................C.......
  .....................C.....................C...........A...
       .....A..........C.....................T...........C........
        ...............C.....................T...........C.........
          .............C.....................C...........A...........
            ...........C.....................C...........A.............
                 ......C.....................C...........A.................
                 ......C.....................C...........A.................
                    ...C.....................G...........A....................
                     ..C.....................T...........C.....................
```

We can examine the aligned reads to determine the number of times each allelic combination (haplotype) occurs.
We call this tally a *typing result*.
The typing result for this example is as follows: the `C,C,A` haplotype is observed 5 times, the `C,T,C` haplotype is observed 3 times, and the `C,G,A` haplotype is observed 1 time.
If we then filter out the `C,G,A` haplotype as erroneous (see below), we can infer a diploid `C,C,A / C,T,C` genotype for this marker.
We call this process *genotype prediction*.

It is helpful to point about a few observations about this example.
- The first aligned read does not span all SNPs at the marker, so it is discarded.
- The third aligned read shows an `A` at the 13th position of the marker reference sequence. Whether this reflects true genetic variation or is a technical artifact resulting from sequencing error, it is ignored by MicroHapulator because it is not one of the three SNPs of interest.
- The `C,G,A` haplotype is only observed once and is likely a false haplotype resulting from sequencing error at the second SNP of interest in the haplotype. When dozens or hundreds (or thousands!) of reads are successfully sequenced and aligned to the marker reference, it is usually simple to distinguish signal (true haplotypes) from noise (false haplotypes resulting from sequencing error). When the depth of sequencing coverage is low, as it is in this example, it can be more difficult to distinguish signal from noise. Determining appropriate per-marker thresholds (detection thresholds and analytical thresholds) for filtering will typically require a non-trivial amount of testing with the laboratory's NGS sequencing instrument(s).

While this mock example is helpful in building intuition about haplotype calling, manual visual examination is not feasible for performing this task on dozens of markers and (potentially) millions of NGS reads.
The MicroHapulator software provides tools that automate haplotype calling and genotype prediction, as well as assist with basic interpretation of the forensic typing result.

## Setup

Two mock scenarios are presented in this demo.
In both scenarios, a number of reference and evidentiary samples have been sequenced on an Illumina MiSeq with a panel targeting 23 microhaplotype markers.
The identifiers for these markers are shown below.

In [ ]:
cat panel.txt

Following the instructions in the [MicroHapulator configuration manual](https://microhapulator.readthedocs.io/en/latest/config.html), configuration files were prepared previously with marker reference sequences, microhaplotype SNP definitions, and haplotype frequencies for the population of interest.
These files are listed as follows.
(The full contents of these files are available HERE (LINK FIXME).)

In [ ]:
ls -1 refr-seqs.fasta marker-defn.tsv frequencies.tsv

One of the initial steps in data analysis is mapping the NGS reads to the marker reference sequences.
This mapping procedure requires the construction of a search index for the reference sequences.
The indexing task only needs to be performed once for any given reference sequence file.

In [ ]:
bwa index refr-seqs.fasta

And then of course, we need to download the NGS reads for our mock scenarios.

In [ ]:
echo FIXME

## Data Preprocessing and Haplotype Calling

Before diving in to our mock scenarios, let us first demonstrate how to compute a typing result.
The paired-end reads generated for each sample by the Illumina MiSeq instrument are stored in paired files, as shown below for the `EVD1` sample.

In [ ]:
ls -1 EVD1-reads-R*.fastq.gz

Because of the size of the target amplicons and the length of each sequenced fragment, we expect most read pairs to have a substantial amount of overlap.
Rather than aligning and probing the paired fragments separately, we will first attempt to merge each read pair into a single sequence.
For this we use the FLASH program.

In [ ]:
flash EVD1-reads-R1.fastq.gz EVD1-reads-R2.fastq.gz --min-overlap=100 --max-overlap=325 --output-prefix=EVD1 --allow-outies

As noted in the FLASH output, we now have merged reads stored in the file `EVD1.extendedFrags.fastq`.
The next step in our workflow is to map the reads to the target amplicon sequences in `refr-seqs.fasta`.
In this notebook we use the `bwa mem` algorithm, but other algorithms such as `bowtie2` would also be appopriate to use here.
We also use `samtools` to convert the plain text alignments in SAM format to sorted, compressed, and indexed read alignments in BAM format.

In [ ]:
bwa mem refr-seqs.fasta EVD1.extendedFrags.fastq | samtools view -b | samtools sort -o EVD1-reads.bam
samtools index EVD1-reads.bam

With the aligned reads stored in `EVD1-reads.bam`, we can now compute the typing result for this sample with MicroHapulator.
In addition to the BAM file containing read alignments, we also need to specify the configuration file containing marker definitions for the 23-plex panel.
We'll store the result in `EVD1-result.json` and then peek at the first several lines of this file.

Due to sequencing errors, some of the haplotypes observed in a typing result will be technical artifacts.
While computing a typing result, the `mhpl8r type` command can also apply naïve static and/or dynamic filters to distinguish true haplotypes from false and determine the genotype of the sample.
We call the filtered typing result a *genotype call*.

In [ ]:
mhpl8r type marker-defn.tsv EVD1-reads.bam --dynamic 0.1 --out EVD1-result.json
cat EVD1-result.json | head -n 38

The data is stored in JavaScript Object Notation (JSON), and includes for each marker a handful of coverage statistics, the quantitative typing result, and the genotype call.
Here we see the first two entries in the file, corresponding to the markers `mh01USC-1pD` and `mh02USC-2pC` which appear to be heterozygous for this sample.

As a final sanity check, it's always critical to examine the *interlocus balance* of a sample.
Are there any markers with a disproportionately high or low number of aligned reads?
It is normal to observe some variation in the number of reads aligned to each marker.
This variation could be due to any combination of factors, such as primer kinetics, off-target amplification, or stochastic effects in sequencing.
With the application of appropriate thresholds, interlocus imbalance shouldn't cause serious problems with analysis and interpretation of a sample except in cases of extreme imbalance or the presence of DNA contributor(s) at very low levels in a sample.

We assess interlocus balance with the `mhpl8r balance` command.

In [ ]:
mhpl8r balance EVD1-result.json

Conveniently, the interlocus balance for this sample is high—artificially high in fact because it's simulated mock data.
We would expect to see more variation in real data coming off an NGS sequencer.
But observing no significant imbalance, we can proceed with our analysis and interpretation.

## Scenario 1

In our first scenario, two evidentiary samples have been collected in the course of a forensic investigation.
The case worker has leveled these samples **EVD1** and **EVD2** and suspects that both have only a single DNA contributor.
We also have a reference sample labeled **REF1** collected from a person of interest in the investigation.
All three samples were assayed with our 23-plex NGS panel, and the reads were stored in the pairs of files: `EVD1-reads-R*.fastq.gz`, `EVD2-reads-R*.fastq.gz`, and `REF1-reads-R*.fastq.gz`.

In [ ]:
ls -1 EVD1-reads-R*.fastq.gz EVD2-reads-R*.fastq.gz REF1-reads-R*.fastq.gz

We have already shown how to merge and align the NGS reads and call haplotypes for a sample with **EVD1**.
Let us repeat the process for **EVD2** and **REF1**.

In [ ]:
flash EVD2-reads-R1.fastq.gz EVD2-reads-R2.fastq.gz --min-overlap=100 --max-overlap=325 --output-prefix=EVD2 --allow-outies
bwa mem refr-seqs.fasta EVD2.extendedFrags.fastq | samtools view -b | samtools sort -o EVD2-reads.bam
samtools index EVD2-reads.bam
mhpl8r type marker-defn.tsv EVD2-reads.bam --dynamic 0.1 --out EVD2-result.json

flash REF1-reads-R1.fastq.gz REF1-reads-R2.fastq.gz --min-overlap=100 --max-overlap=325 --output-prefix=REF1 --allow-outies
bwa mem refr-seqs.fasta REF1.extendedFrags.fastq | samtools view -b | samtools sort -o REF1-reads.bam
samtools index REF1-reads.bam
mhpl8r type marker-defn.tsv REF1-reads.bam --dynamic 0.1 --out REF1-result.json

And let's also assess interlocus balance.

In [ ]:
mhpl8r balance EVD2-result.json
mhpl8r balance REF1-result.json

With no concerns about interlocus balance, we can move on to forensic interpretation.
Forensic interpretation refers to the process of determining the conclusions, if any, that can be responsibly drawn by comparing two or more DNA profiles.
How confident can one be that two single-source profiles came from the same individual?
How confident can one be that one single-source profile is a contributor to a second mixture profile?
A variety of approaches exist for addressing these types of questions, some very simple and some very complex.
MicroHapulator implements a handful of tools for simple forensic interpretation tasks.
It also supports the export of typing results to a format that can be used by state-of-the-art probabilistic genotyping (probgen) programs.
Probgen is beyond the scope of this tutorial, but we will demonstrate the basic interpretation capabilities provided by MicroHapulator.

One question often encountered during a study or investigation is the number of DNA contributors in an evidentiary sample (or sometimes even a sample collected from a person of interest).
A single-source profile will have at most two distinct alleles at any given marker, one from each parental haplotype.
However, it's likewise possible that profile with two contributors will *also* have no more than two distinct alleles, even if the contributing genotypes are different (e.g. contributor A may be homozygous for one allele and contributor B homozygous for another allele).
But chances are that two or more DNA contributors will result in three or more alleles for at least a *small* number of markers.
We can use this observation to estimate the minimum number of contributors for a profile.

The `mhpl8r contrib` command implements a procedure to scan a typing result to determine the maximum number of alleles $N_{\text{al}}$ present at any single locus. From this, it can calculate the minimum number of DNA contributors $C_{\text{min}}$ as follows.

$$
C_{\text{min}} = \left\lceil\frac{N_{\text{al}}}{2}\right\rceil
$$

We begin by applying this to the three profiles in our mock scenario.

In [ ]:
mhpl8r contrib EVD1-result.json
mhpl8r contrib EVD2-result.json
mhpl8r contrib REF1-result.json

In all three profiles, none of the 23 markers has evidence for more than a single contributor.
It would thus be reasonable to pursue additional interpretation under the assumption that these are all single-source samples.

Depending on the details of the investigation, it might be necessary to investigate whether the two evidentiary came from the same individual, or whether one of the the evidentiary samples matches a reference sample collected from a person of interest.
The most basic approach to addressing this question is to examine a pair of profiles and determine, marker by marker, whether there are any differences between the genotypes.
Given two MicroHapulator typing results, the `mhpl8r diff` command will print any alleles that are present in one profile but not the other.

In [ ]:
mhpl8r diff EVD1-result.json REF1-result.json

In [ ]:
mhpl8r diff EVD1-result.json EVD2-result.json

The first result shows that there are no differences between the **EVD1** profile and the **REF1** profile, so there is already strong evidence that these were derived from the same individual.

The second result shows numerous differences between **EVD1** and **EVD2**.
It's not uncommon to see minor discrepancies between different samples originating from the same individual—variability in laboratory processing or sample degradation could explain some of these differences.
But if this were the case we would not typically expect to see discordant alleles at almost every marker.
The much more likely explanation in this case is that **EVD1** and **EVD2** come from different individuals.

But beyond a basic check for their presence or absence, an exhaustive listing of discordant alleles isn't very helpful (except perhaps for troubleshooting purposes).
What we *really* want is a quantitative measure of our confidence in a match between two profiles.
We derive this measure by assessing the likelihood of two propositions—two competing hypotheses or explanations for the data.
Depending on the details of the investigation, we might formulate the likelihood ratio (LR) test as something like the following.

- $H_p$: **REF1** and **EVD1** originated from the same individual
- $H_d$: **REF1** and **EVD1** originated from two unrelated individuals in the population

We then compute the ratio of the two hypotheses' probabilities as $LR = \frac{P(H_p)}{P(H_d)}$.
Large LR values are strong evidence in favor of $H_p$, small LR values support $H_d$, and LR values close to 1.0 are inconclusive.

The probability $P(H_p) = \epsilon^R$, where $\epsilon$ is a per-marker rate of genotyping error (default: 0.001) and $R$ is the number of markers with discordant alleles between samples.
The probability $P(H_d)$ is the random match probability (RMP) of the profile, which is essentially the product of the observed haplotype frequencies in the population.
Note that in cases of a perfect profile match, $P(H_p) = 1$ and the LR is then simply the reciprocal of the RMP.

We can use `mhpl8r prob` both to compute the RMP for the evidence and to perform the LR test as formulated above.

In [ ]:
mhpl8r prob frequencies.tsv EVD1-result.json
mhpl8r prob frequencies.tsv EVD1-result.json REF1-result.json

The result is a very large LR of $7.18 \times 10^{22}$, lending strong support to $H_p$ over $H_d$, consistent with our earlier observations.

Now let's consider what would happen if, disregarding our earlier observations, we had formulated the LR test as follows.

- $H_p$: **REF2** and **EVD1** originated from the same individual
- $H_d$: **REF2** and **EVD1** originated from two unrelated individuals in the population

In [ ]:
mhpl8r prob frequencies.tsv EVD2-result.json
mhpl8r prob frequencies.tsv EVD2-result.json REF1-result.json

Here we see a very different result.
The RMP for **EVD2** is of similar magnitude to that of **EVD1**, but the LR test statistic is very small.
A tremendous amount of error would be required if **EVD2** and **REF1** were from the same individual—much more likely is that these samples originated from two unrelated individuals.

## Scenario 2

In this scenario, the case worker has collected an evidentiary sample (**EVD3**) in the course of a forensic investigation, and there is some suspicion that this sample has multiple DNA contributors.
We have also collected reference samples from three persons of interest in the investigation, labeled **REF2**, **REF3**, and **REF4**.
As in the previous scenario, all four samples have been sequenced with our 23-microhap NGS panel.
Reads are available in the following files.

In [ ]:
ls -1 EVD3-reads-R*.fastq.gz REF2-reads-R*.fastq.gz REF3-reads-R*.fastq.gz REF4-reads-R*.fastq.gz

As before, we will merge the read pairs, align the merged reads to the reference sequences, and compute a typing result for each sample.

In [ ]:
flash EVD3-reads-R1.fastq.gz EVD3-reads-R2.fastq.gz --min-overlap=100 --max-overlap=325 --output-prefix=EVD3 --allow-outies
bwa mem refr-seqs.fasta EVD3.extendedFrags.fastq | samtools view -b | samtools sort -o EVD3-reads.bam
samtools index EVD3-reads.bam
mhpl8r type marker-defn.tsv EVD3-reads.bam --dynamic 0.1 --out EVD3-result.json

flash REF2-reads-R1.fastq.gz REF2-reads-R2.fastq.gz --min-overlap=100 --max-overlap=325 --output-prefix=REF2 --allow-outies
bwa mem refr-seqs.fasta REF2.extendedFrags.fastq | samtools view -b | samtools sort -o REF2-reads.bam
samtools index REF2-reads.bam
mhpl8r type marker-defn.tsv REF2-reads.bam --dynamic 0.1 --out REF2-result.json

flash REF3-reads-R1.fastq.gz REF3-reads-R2.fastq.gz --min-overlap=100 --max-overlap=325 --output-prefix=REF3 --allow-outies
bwa mem refr-seqs.fasta REF3.extendedFrags.fastq | samtools view -b | samtools sort -o REF3-reads.bam
samtools index REF3-reads.bam
mhpl8r type marker-defn.tsv REF3-reads.bam --dynamic 0.1 --out REF3-result.json

flash REF4-reads-R1.fastq.gz REF4-reads-R2.fastq.gz --min-overlap=100 --max-overlap=325 --output-prefix=REF4 --allow-outies
bwa mem refr-seqs.fasta REF4.extendedFrags.fastq | samtools view -b | samtools sort -o REF4-reads.bam
samtools index REF4-reads.bam
mhpl8r type marker-defn.tsv REF4-reads.bam --dynamic 0.1 --out REF4-result.json

As always, we assess interlocus balance.

In [ ]:
mhpl8r balance EVD3-result.json
mhpl8r balance REF2-result.json
mhpl8r balance REF3-result.json
mhpl8r balance REF4-result.json

We now want to examine the evidentiary sample and investigate the presence of multiple DNA contributors.

In [ ]:
mhpl8r contrib profile-EVD3.json

In this sample, we observe three loci that have evidence for at least three DNA contributors (i.e. five or more distinct haplotypes).
We can then test whether any of the reference samples is a contributor to the evidentiary mixture.
MicroHapulator implements a simple containment test to investigate this question.
In brief, the containment test examines each marker and determines whether the genotype in a profile of interest is compatible with the genotype of a mixture profile.
For example, if a reference sample **REF99** has a `A,C,T / A,T,T` genotype at a marker and an evidentiary mixture sample **EVD99** has a `A,C,G / A,C,T / A,T,T` genotype, **REF99** is compatible, e.g. a plausible contributor to **EVD99**.
On the other hand, if the **REF99** genotype at a different marker is `G,G,A,T / C,G,T,T` and the **EVD99** genotype is `G,G,A,T / G,G,T,T`, **REF99** is not a plausible contributor to **EVD99**.
In the end, the containment test reports the percentage of markers in one profile with compatible genotypes to another (mixture) profile.

We can use the `mhpl8r contrib` command to perform the containment test for **REF2**.

In [ ]:
mhpl8r contain EVD3-result.json REF2-result.json

This tells us that 100% of the haplotypes observed in **REF2** are present in **EVD3**, suggesting that **REF2** is a plausible contributor to **EVD3**.
What can we say about **REF3** and **REF4**?

In [ ]:
mhpl8r contain EVD3-result.json REF3-result.json
mhpl8r contain EVD3-result.json REF4-result.json

Only about FIXME% of the alleles in both of these samples are present in **EVD3**, suggesting that they are unlikely to be contributors to the sample.

As a final note, it important acknowledge several factors that can influence the the containment statistic.
It is not always possible to develop thresholds that eliminate all errors and detects all true haplotypes.
This is especially the case for low-input samples or mixtures with an imbalance between major and minor contributors.
MicroHapulator's containment test does not account for allelic drop-in (false positives) or drop-out (false negatives).
We also note that many haplotypes from non-contributors will be present in a mixture simply by chance, and there is no simple theoretical threshold we can apply to distinguish between "true" and "false" containment.
The containment test is a basic tool that provides a quick check to guide an investigation.
Probabilistic genotyping (probgen) methods, on the other hand, provide for a much more robust interpretation of forensic propositions, with models for drop-in and drop-out evaluated in a maximum likelihood framework.
Probgen is beyond the scope of this tutorial, but the use of probgen tools such as LRMix Studio or EuroForMix is recommended for definitive interpretation of DNA mixtures.
